In [1]:
import sklearn
import math
import pandas
import numpy
from sklearn.linear_model import LinearRegression

In [2]:
df = pandas.DataFrame({"x": numpy.random.rand(100)})
df["y"] = -df["x"] * df["x"] + 4
df["z"] = df["y"] * 2 + df["x"]

In [3]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

space = [
    Integer(0, 1, name="intercept"),
    Real(0, 1, "uniform", name="dummy"),
    Real(1, 5, "uniform", name="scale"),    
]

In [4]:
from skopt import gp_minimize
from skopt import forest_minimize

def opt_fit(space):
    @use_named_args(space)
    def my_objective(**params):
        obj = _objective(**params)
        return -obj
    gp = forest_minimize(func=my_objective, dimensions=space)
    return gp
    
def _objective(**params):
    print(params)
    df2 = df.copy()
    lm = LinearRegression(fit_intercept=params["intercept"])
    df2["z"] = df2["z"] + numpy.random.rand(len(df2)) * params["scale"] + params["dummy"]
    lm.fit(df2[["x", "y"]], df2["z"])
    obj = lm.score(df2[["x", "y"]], df2["z"])
    return obj

In [5]:
g = opt_fit(space)

{'intercept': 0, 'dummy': 0.5613264206512585, 'scale': 1.2040938051405932}
{'intercept': 1, 'dummy': 0.8265556066777213, 'scale': 3.2372944990179318}
{'intercept': 1, 'dummy': 0.4299074707353995, 'scale': 4.212118883412312}
{'intercept': 1, 'dummy': 0.914679403364216, 'scale': 3.676407661876897}
{'intercept': 1, 'dummy': 0.09687277295877364, 'scale': 1.2088068519475939}
{'intercept': 1, 'dummy': 0.9816371482400555, 'scale': 1.2358602826775478}
{'intercept': 1, 'dummy': 0.38140242210479536, 'scale': 4.925074236050078}
{'intercept': 0, 'dummy': 0.4370978861199155, 'scale': 3.2419187577380613}
{'intercept': 0, 'dummy': 0.17695224964012005, 'scale': 1.2569306149095985}
{'intercept': 0, 'dummy': 0.5593216909566082, 'scale': 3.3530428428929504}
{'intercept': 1, 'dummy': 0.2510066780190568, 'scale': 1.1574543186333446}
{'intercept': 1, 'dummy': 0.056418427282320174, 'scale': 1.5546157147007356}
{'intercept': 1, 'dummy': 0.3833996795047273, 'scale': 1.0498274884836634}
{'intercept': 1, 'dummy'

In [6]:
g.fun

-0.6516630946682641

In [7]:
g.x

[0, 0.44215255958028, 1.0729714092421192]

In [8]:
g

          fun: -0.6516630946682641
    func_vals: array([-0.49803875, -0.19442909, -0.10619416, -0.23189763, -0.51030476,
       -0.46597982, -0.00500006, -0.15463062, -0.49203679, -0.09449031,
       -0.59814673, -0.37816246, -0.59710501, -0.43687564, -0.61891348,
       -0.47156946, -0.36893925, -0.48018701, -0.51077146, -0.42908682,
       -0.48046447, -0.57116858, -0.57410762, -0.51501258, -0.62401959,
       -0.65166309, -0.61029666, -0.56539365, -0.51652828, -0.63478479,
       -0.54257169, -0.55655677, -0.52862076, -0.52704786, -0.5776939 ,
       -0.48644662, -0.53047503, -0.30024505, -0.59554383, -0.55682499,
       -0.4904214 , -0.60870113, -0.53980066, -0.40388104, -0.56714488,
       -0.58949176, -0.55226545, -0.56641532, -0.47386841, -0.62949178,
       -0.47939073, -0.56227409, -0.51673353, -0.49329585, -0.51046931,
       -0.60243105, -0.64330725, -0.61361902, -0.50626513, -0.59547443,
       -0.64137272, -0.50516527, -0.64727617, -0.59599981, -0.59191448,
       -0.4998